In [2]:
! . /usr/local/bin/thisroot.sh
import ROOT
from ROOT import TFile,TH1,TH1F,TCanvas,TLine,TLegend,TPad
from ROOT import RooFit

Welcome to JupyROOT 6.10/08


In [3]:
#! ls ../rootfiles/X4140*.root

In [4]:
rootfile ="../rootfiles/X4140_Base_KRe_MuRef_NoQual_Sidebands5-7_NP3.0_B0Cuts_CW5.15-5.55_BDCAdd.root"
rootfile = "../rootfiles/X4140_Base_Side4-6_AllData.root"
#rootfile ="../rootfiles/X4140_MuMuKK_KRe_MuRef_Sidebands5-7_NP3.0_B0Cuts_CW5.15-5.55.root"
#histname = "Xcand_histo_hlt8_cw_nonprompt_cosalpha"
#histname = "Phi_hist_all_cw_all_cosalpha"
#histname = "PhiMassHisto"
histname = "CW_PhiMass_NoM"
#histname = "Xcand_histo_DM_any_cw_nonprompt_cosalpha"
inputfile  = TFile(rootfile,"READ")
hist = inputfile.Get(histname)
c = TCanvas("canvas","canvas",1200,800) ;
hist.SetName("CW_Phi_NonPrompt")
hist.SetTitle("CW_Phi_NonPrompt")

In [5]:
from ROOT import RooRealVar,RooAbsPdf,RooChebychev,RooExponential,RooGaussian
from ROOT import RooAbsPdf,RooPlot,RooAddPdf,RooDataHist,RooArgSet,RooArgList
from ROOT import RooVoigtian
from ROOT import kGreen,kRed,kBlack,kBlue,kDashed,kDotted,kMagenta
from ROOT.RooFit import Components,LineColor,LineStyle,Name,Normalization,Layout,Format,Label,Parameters,Range,Title

In [6]:
massmin = 1.020-0.03
massmax = 1.020+0.03
phimean = 1.020
massbins = (massmax - massmin)/hist.GetBinWidth(2)
mass = RooRealVar("mass","M(KK)[GeV]",massmin,massmax)
mass.setBins(int(massbins))

In [7]:
hist.GetXaxis().SetRangeUser(massmin,massmax)
hist.Draw()
plotname = "plots/phiplot_" + histname
c.SaveAs(plotname  + ".png")
c.SaveAs(plotname  + ".eps")
c.SaveAs(plotname  + ".pdf")

Info in <TCanvas::Print>: png file plots/phiplot_CW_PhiMass_NoM.png has been created
Info in <TCanvas::Print>: eps file plots/phiplot_CW_PhiMass_NoM.eps has been created
Info in <TCanvas::Print>: pdf file plots/phiplot_CW_PhiMass_NoM.pdf has been created


In [8]:
mean = RooRealVar("mean","mean of gaussian",phimean,phimean-0.005,phimean+0.005);
sigma = RooRealVar("sigma","width of gaussian",0.0013);
gamma = RooRealVar("gamma","gamma of bw",0.004253,0.001,0.01);

a0 = RooRealVar("a0","a0",0.001,-1.,1.)
a1 = RooRealVar("a1","a1",0.001,-0.5,0.5)
a2 = RooRealVar("a2","a2",-0.00001,-2.,2.)
a3 = RooRealVar("a3","a3",0.0)#
a4 = RooRealVar("a4","a4",0.0,-0.1,0.1)
a5 = RooRealVar("a5","a5",0.0,-0.025,0.05)
a6 = RooRealVar("a6","a6",0.0,-0.001,0.001)

aset = RooArgList(a0,a1,a2,a3,a4,a5)
nSig = RooRealVar("nSig","nSig",5E5,0.,5.0E6)
nBkg = RooRealVar("nBkg","nBkg",2E6,0.,5.0E6)

In [9]:
cheb = RooChebychev("cheb","Background",mass,aset)
#gauss = RooGaussian("gauss","gaussian PDF ",mass,mean,sigma)
signal = RooVoigtian("signal","signal",mass,mean,gamma,sigma)

tot = RooAddPdf("tot","g+cheb",RooArgList(signal,cheb),RooArgList(nSig,nBkg))

In [10]:
masslist = RooArgList(mass)
dh = RooDataHist("dh","dh",masslist,hist)
numEvts = dh.sum(False)
print numEvts

240641.0
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(dh): fit range of variable mass expanded to nearest bin boundaries: [0.99,1.05] --> [0.99,1.05]


In [11]:
tot.fitTo(dh)

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:NumericIntegration -- RooRealIntegral::init(signal_Int[mass]) using numeric integrator RooIntegrator1D to calculate Int(mass)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:NumericIntegration -- RooRealIntegral::init(signal_Int[mass]) using numeric integrator RooIntegrator1D to calculate Int(mass)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (signal,cheb)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0           1.00000e-03  2.00000e-01   -1.00000e+00  1.00000e+00
     2 a1           1.00000e-03  1.00000e-01   -5.00000e-01  5.00000e-01
     3 a2          -1.00000e-05  4.00000e-01   -2.00000e+00  2.00000e+00
     4 a4           0.00000e+

In [12]:
plotmax = hist.GetMaximum()*1.05
sidesigma = sigma.getValV()
sidesigma = gamma.getValV()
leftlowside = -6.*sidesigma + mean.getValV()
leftupside = -4.*sidesigma + mean.getValV()
rightlowside = +4.*sidesigma + mean.getValV()
rightupside = +6.*sidesigma + mean.getValV()

signallow = -3.*sidesigma + mean.getValV()
signalup = +3.*sidesigma + mean.getValV()

print "Side Sigma"
print "%.5e" % sidesigma
print "Left Side"
print "%.5f - %.5f" % (leftlowside,leftupside)
print mean.getValV()

Side Sigma
4.33243e-03
Left Side
0.99359 - 1.00226
1.01958952169


In [13]:
for param in [False,True]:
    cmass = TCanvas("canvas","canvas",1200,1200) ;
    JPsi_mass = 3.096916
    Phi_mass = 1.019455
    th_mass = JPsi_mass + Phi_mass

    pullpad = TPad("pullpad","pullpad",0.0,0.05,1.0,0.33)
    plotpad = TPad("histopad","histopad",0.0,0.35,1.0,1.0)
    plotpad.SetFillStyle(4004)
    pullpad.SetFillStyle(4004)
    plotpad.Draw()
    pullpad.Draw()

    massFrame = mass.frame()
    massFrame.SetTitle("Phi signal - CC - NonPrompt")
    dh.plotOn(massFrame,Name("Data"))
    tot.plotOn(massFrame,Name("Pdf"))
    massFrame.GetYaxis().SetTitleOffset(1.3)
    #gauss.plotOn(massFrame,LineColor(kGreen),LineStyle(kDashed),Normalization((sFrac.getValV()*numEvts)/(numEvts)))
    #cheb.plotOn(massFrame,LineColor(kMagenta),LineStyle(kDotted),Normalization(((1.0-sFrac.getValV())*numEvts)/(numEvts)))
    signal.plotOn(massFrame,LineColor(kGreen),LineStyle(kDashed),Normalization((nSig.getValV())/(numEvts)))
    cheb.plotOn(massFrame,LineColor(kGreen),LineStyle(kDashed),Normalization((nBkg.getValV())/(numEvts)))
    if param:
        tot.paramOn(massFrame,Layout(0.60,0.99,0.75))
    plotpad.cd()
    massFrame.Draw()

    linelow = TLine(leftlowside,0.0,leftlowside,plotmax)
    linemidlow = TLine(leftupside,0.0,leftupside,plotmax)
    linemidup = TLine(rightlowside,0.0,rightlowside,plotmax)
    lineup = TLine(rightupside,0.0,rightupside,plotmax)

    linesiglow = TLine(signallow,0.0,signallow,plotmax)
    linesigup = TLine(signalup,0.0,signalup,plotmax)

    linelow.SetLineColor(kRed)
    linemidlow.SetLineColor(kRed)
    linemidup.SetLineColor(kRed)
    lineup.SetLineColor(kRed)

    linesiglow.SetLineColor(kGreen)
    linesigup.SetLineColor(kGreen) 

    linelow.SetLineWidth(2)
    linemidlow.SetLineWidth(2)
    linemidup.SetLineWidth(2)
    lineup.SetLineWidth(2)

    linesiglow.SetLineWidth(2)
    linesigup.SetLineWidth(2) 
    
    if not param:
        linelow.Draw()
        linemidlow.Draw()
        linemidup.Draw()
        lineup.Draw()

        linesiglow.Draw()
        linesigup.Draw()


    hpull = massFrame.pullHist("Data","Pdf")
    pullframe = mass.frame(Title("Pull Distribution"))
    #pullframe.GetXaxis().SetTitleSize(0.04)
    #pullframe.GetYaxis().SetTitleSize(0.03)
    ROOT.gStyle.SetTitleFontSize(0.07)
    pullpad.cd()

    pullframe.addPlotable(hpull,"P")
    pullframe.Draw()
    upLine = TLine(massmin,3.0,massmax,3.0)
    upLine.SetLineColor(kRed)
    upLine.SetLineStyle(kDashed)
    upLine.SetLineWidth(2)
    dwLine = TLine(massmin,-3.0,massmax,-3.0)
    dwLine.SetLineColor(kRed)
    dwLine.SetLineStyle(kDashed)
    dwLine.SetLineWidth(2)
    dwLine.Draw()
    upLine.Draw()
    plotname = "plots/phi_" 
    
    if param:
        plotname += "paramon_"
        
    plotname += histname
    cmass.SaveAs(plotname  + ".png")
    #cmass.SaveAs(plotname  + ".eps")
    cmass.SaveAs(plotname  + ".pdf")

[#1] INFO:NumericIntegration -- RooRealIntegral::init(signal_Int[mass]) using numeric integrator RooIntegrator1D to calculate Int(mass)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(signal_Int[mass]) using numeric integrator RooIntegrator1D to calculate Int(mass)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(signal_Int[mass]) using numeric integrator RooIntegrator1D to calculate Int(mass)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(signal_Int[mass]) using numeric integrator RooIntegrator1D to calculate Int(mass)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file plots/phi_CW_PhiMass_NoM.png has been created
Info in <TCanvas::Print>: pdf file plots/phi_CW_PhiMass_NoM.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file plots/phi_paramon_CW_PhiMass_NoM.png has been created
Info in <TCanvas::Print>: pdf file plots/phi_paramon_CW_PhiMass_NoM.pdf has been created


In [5]:
mass.setRange("signalrange",signallow,signalup)
mass.setRange("sideleftrange",leftlowside,leftupside)
mass.setRange("siderightrange",rightlowside,rightupside)
signalIntegralBkg = cheb.analyticalIntegral(cheb.getAnalyticalIntegral(RooArgSet(mass),RooArgSet(mass)),"signalrange")
leftsideIntegralBkg = cheb.analyticalIntegral(cheb.getAnalyticalIntegral(RooArgSet(mass),RooArgSet(mass)),"sideleftrange")
rightsideIntegralBkg = cheb.analyticalIntegral(cheb.getAnalyticalIntegral(RooArgSet(mass),RooArgSet(mass)),"siderightrange")

NameError: name 'mass' is not defined

In [14]:
totIntegralBkg = cheb.analyticalIntegral(cheb.getAnalyticalIntegral(RooArgSet(mass),RooArgSet(mass)))
print signalIntegralBkg/totIntegralBkg

0.444662371894


In [15]:
#sigBkgEvts = signalIntegralBkg/totIntegralBkg*((1.0-sFrac.getValV())*numEvts)
#sidBkgEvts = (leftsideIntegralBkg+rightsideIntegralBkg)/totIntegralBkg*((1.0-sFrac.getValV())*numEvts)
sigBkgEvts = signalIntegralBkg/totIntegralBkg*((nBkg.getValV()))
sidBkgEvts = (leftsideIntegralBkg+rightsideIntegralBkg)/totIntegralBkg*((nBkg.getValV()))

In [16]:
print sigBkgEvts

81613.7170948


In [17]:
print sidBkgEvts

50088.0502348


In [18]:
ratio = sigBkgEvts/sidBkgEvts
print ratio

1.62940495212


In [6]:
theRatio = 1.62940495212 #inputfile.ls()

In [7]:
siderootfile = "../rootfiles/X4140_Base_AllDatasets.root"
sidefile  = TFile(siderootfile,"READ")
signalCW = sidefile.Get("CW_Mass_Signal_NoM")
sideCW_L = sidefile.Get("CW_Mass_L_Side_NoM")
sideCW_R = sidefile.Get("CW_Mass_R_Side_NoM")
print sideCW_R.GetEntries()
print sideCW_L.GetEntries()
print signalCW.GetEntries()
notrebin = True

33962.0
17272.0
135371.0


In [8]:
if notrebin:
    signalCW.Rebin(10)
    sideCW_L.Rebin(10)
    sideCW_R.Rebin(10)
    notrebin = False

In [9]:
side = sideCW_R.Clone()
side.Add(sideCW_L)
side.Scale(theRatio)
sideCW_L.Scale(theRatio)
sideCW_R.Scale(theRatio)

In [12]:
cB0 = TCanvas("cB0","cB0",1200,800)
signalCW.SetFillColor(kBlue)
signalCW.SetName("B_{0}^{s} Candidates Mass - CC")
signalCW.SetTitle("B_{0}^{s} Candidates Mass - CC; M(KK#mu#mu)[GeV]; candidates/" + str(signalCW.GetBinWidth(2)*1000)+ "MeV")
signalCW.GetYaxis().SetTitleOffset(1.3)
signalCW.SetMarkerColor(kBlue)
signalCW.SetFillStyle(3002)
signalCW.SetMarkerStyle(ROOT.kFullCircle)
signalCW.SetMarkerSize(0.5)
signalCW.SetLineColor(kBlue)

sideCW_L.SetFillColor(kRed)
#sideCW_L.SetMarkerColor(kBlack)
sideCW_L.SetFillStyle(3002)
sideCW_L.SetMarkerStyle(ROOT.kFullCircle)
sideCW_L.SetMarkerSize(0.5)
sideCW_L.SetLineColor(kBlack)

sideCW_R.SetFillColor(kGreen)
#sideCW_R.SetMarkerColor(kBlack)
sideCW_R.SetMarkerStyle(ROOT.kFullCircle)
sideCW_R.SetMarkerSize(0.5)
sideCW_R.SetLineColor(kBlack)
sideCW_R.SetFillStyle(3002)

signalCW.Draw("EBar")
side.SetFillColor(kGreen)
#sideCW_R.SetMarkerColor(kBlack)
side.SetMarkerStyle(ROOT.kFullCircle)
side.SetMarkerSize(0.5)
side.SetLineColor(kBlack)
side.SetFillStyle(3002)
#sideCW = sideCW_L.Clone()
#sideCW.Add(sideCW_R,+1.0)
#sideCW.SetFillColor(kRed)
#sideCW.SetFillStyle(3002)
#sideCW.Scale(2.0)
#sideCW.Draw("SAMEBar")
#sideCW_R.Draw("E0SAMEBar")
#sideCW_L.Draw("E0SAMEBar")
side.Draw("E0SAMEBar")

legend = TLegend(0.75,0.45,0.99,0.75)
legend.AddEntry(signalCW,"Signal region","f")
#legend.AddEntry(signalCW,"Signal region (-3.0#sigma,+3.0#sigma)","f")
#legend.AddEntry(sideCW_R,"R-sideband    (+4.0#sigma,+6.0#sigma)","f")
#legend.AddEntry(sideCW_L,"L-sideband    (-6.0#sigma,-4.0#sigma)","f")
legend.AddEntry(side,"Sidebands ","f")
legend.Draw()
cB0.SaveAs("plots/" + signalCW.GetName() + "_sidebands.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cB0
Info in <TCanvas::Print>: png file plots/B_{0}^{s} Candidates Mass - CC_sidebands.png has been created


In [13]:
ROOT.gStyle.SetOptStat(0)
b0SideSub = signalCW.Clone()
b0SideSub.SetTitle("B_{0}^{s} Mass - #phi sides subtracted; M(KK#mu#mu)[GeV]; candidates/" + str(signalCW.GetBinWidth(2)*1000)+ "MeV")
b0SideSub.Add(sideCW_L,-1.0)
b0SideSub.Add(sideCW_R,-1.0)

True

In [9]:
b0SideSub.SetFillColor(4004)
b0SideSub.SetFillStyle(3002)
b0SideSub.SetMarkerStyle(ROOT.kFullCircle)
b0SideSub.SetMarkerColor(kBlack)
b0SideSub.SetMarkerSize(0.8)
b0SideSub.SetLineColor(kBlack)
b0SideSub.Draw("E0")

linezero = TLine(b0SideSub.GetBinCenter(1),0.0,b0SideSub.GetBinCenter(b0SideSub.GetNbinsX()),0.0)
linezero.SetLineColor(kRed)
linezero.SetLineWidth(2)
linezero.SetLineStyle(kDotted)
linezero.Draw()
cB0.SaveAs("plots/" + signalCW.GetName() + "_subtracted.png")

Info in <TCanvas::Print>: png file plots/B_{0}^{s} Candidates Mass - CC_subtracted.png has been created


In [10]:
cSubFit = TCanvas("cSubFit","cSubFit",1200,800)

histname = b0SideSub.GetName()
massmin = 5.2
massmax = 5.55
massbins = (massmax - massmin)/0.005
mass = RooRealVar("mass","M(#mu#muKK)[GeV]",massmin,massmax)
mass.setBins(int(massbins))

mean = RooRealVar("mean","mean of gaussian",5.38,5.31,5.41);
sigma1 = RooRealVar("sigma1","width of gaussian1",0.002,0.0005,0.05);
sigma2 = RooRealVar("sigma2","width of gaussian2",0.004,0.004,0.01);

a0 = RooRealVar("a0","a0",0.001,-1.,1.)
a1 = RooRealVar("a1","a1",0.001,-0.5,0.5)
a2 = RooRealVar("a2","a2",-0.00001,-2.,2.)
a3 = RooRealVar("a3","a3",-0.000001,-0.1,0.1)
a4 = RooRealVar("a4","a4",-0.000001,-2.,2.)
a5 = RooRealVar("a5","a5",-0.000001)
a6 = RooRealVar("a6","a6",-0.000001,-0.01,0.01)

aset = RooArgList(a0,a1)#,a2)#,a3)
gaussFrac = RooRealVar("sig1frac","fraction of component 1 in signal",0.3,0.0,1.0)
nSig = RooRealVar("nSig","nSig",55000,0.,10E6)
nBkg = RooRealVar("nBkg","nBkg",55000,0.,10E6)

cheb = RooChebychev("cheb","Background",mass,aset)
gauss1 = RooGaussian("gauss1","gaussian PDF 1",mass,mean,sigma1)
gauss2 = RooGaussian("gauss2","gaussian PDF 2",mass,mean,sigma2)

model  = RooAddPdf("model","g1+g2",gauss1,gauss2,gaussFrac)
tot = RooAddPdf("tot","g1+g2+cheb",RooArgList(model,cheb),RooArgList(nSig,nBkg))
#tot = model
masslist = RooArgList(mass)
dh = RooDataHist("dh","dh",masslist,b0SideSub);

tot.fitTo(dh)

massFrame = mass.frame()
massFrame.SetTitle("#mu#muKK mass spcetrum - CC")
massFrame.GetYaxis().SetTitleOffset(1.35)
dh.plotOn(massFrame)
tot.plotOn(massFrame)
gauss2.plotOn(massFrame,LineColor(kRed),LineStyle(kDashed),Normalization(gaussFrac.getValV()*nSig.getValV()/(nBkg.getValV()+nSig.getValV())))
gauss1.plotOn(massFrame,LineColor(kGreen),LineStyle(kDashed),Normalization((1.0-gaussFrac.getValV())*nSig.getValV()/(nBkg.getValV()+nSig.getValV())))
cheb.plotOn(massFrame,LineColor(kMagenta),LineStyle(kDotted),Normalization(nBkg.getValV()/(nBkg.getValV()+nSig.getValV())))
tot.paramOn(massFrame,Layout(0.60,0.99,0.9))
massFrame.Draw()

plotname = "plots/b0test_param_" + histname
cSubFit.SaveAs(plotname  + "Fit.png")
cSubFit.SaveAs(plotname  + "Fit.eps")
cSubFit.SaveAs(plotname  + "Fit.pdf")

[#1] INFO:DataHandling -- RooDataHist::adjustBinning(dh): fit range of variable mass expanded to nearest bin boundaries: [5.2,5.55] --> [5.2,5.55]
[#0] WARNING:InputArguments -- RooAbsPdf::fitTo(tot) WARNING: a likelihood fit is request of what appears to be weighted data. 
       While the estimated values of the parameters will always be calculated taking the weights into account, 
       there are multiple ways to estimate the errors on these parameter values. You are advised to make an 
       explicit choice on the error calculation: 
           - Either provide SumW2Error(kTRUE), to calculate a sum-of-weights corrected HESSE error matrix 
             (error will be proportional to the number of events)
           - Or provide SumW2Error(kFALSE), to return errors from original HESSE error matrix
             (which will be proportional to the sum of the weights)
       If you want the errors to reflect the information contained in the provided dataset, choose kTRUE. 
       If yo

Info in <TCanvas::Print>: png file plots/b0test_param_B_{0}^{s} Candidates Mass - CCFit.png has been created
Info in <TCanvas::Print>: eps file plots/b0test_param_B_{0}^{s} Candidates Mass - CCFit.eps has been created
Info in <TCanvas::Print>: pdf file plots/b0test_param_B_{0}^{s} Candidates Mass - CCFit.pdf has been created
